# Embedding samples, storing them in database for future use, and prediction testing

In [4]:
path="C:\\Users\\srish\\Downloads\\archive\\SOCOFing\\Real"

In [ ]:
import psycopg2


def connect():
    conn = psycopg2.connect('REDACTED') # Replace with your connection string

    query_sql = 'SELECT VERSION()'

    cur = conn.cursor()
    return cur, conn




# Embedding and Storing

In [2]:
import os
import numpy as np
import torch
import torch.nn.functional as F
from PIL import Image
from torchvision import transforms
import psycopg2
from pgvector.psycopg2 import register_vector
from psycopg2.extras import execute_values

In [5]:



# -----------------------------------------
# 1) Recreate your metric-learning model
# -----------------------------------------
import torch.nn as nn
from torchvision.models import resnet18, ResNet18_Weights

class FingerprintNet(nn.Module):
    def __init__(self, embedding_dim=128):
        super().__init__()
        self.backbone = resnet18(weights=ResNet18_Weights.DEFAULT)
        self.backbone.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        num_ftrs = self.backbone.fc.in_features
        self.backbone.fc = nn.Linear(num_ftrs, embedding_dim)

    def forward(self, x):
        return F.normalize(self.backbone(x), p=2, dim=1)

# -----------------------------------------
# 2) Load your fine-tuned model
# -----------------------------------------
MODEL_PATH = "D:\\Biometric attendance\\fingerprint_model.pth"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = FingerprintNet(embedding_dim=128).to(device)
state = torch.load(MODEL_PATH, map_location=device, weights_only=False)
model.load_state_dict(state)
model.eval()

# -----------------------------------------
# 3) Precompute gallery embeddings
# -----------------------------------------
DATA_DIR = path
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# gather all BMP paths
gallery_paths = []
for root, _, files in os.walk(DATA_DIR):
    for f in files:
        if f.lower().endswith(".bmp"):
            gallery_paths.append(os.path.join(root, f))
gallery_paths.sort()

# compute embeddings
gallery_records = []  # list of (file_id, [128 floats])
with torch.no_grad():
    for path in gallery_paths:
        img = Image.open(path).convert("L")
        t   = transform(img).unsqueeze(0).to(device)
        emb = model(t).cpu().numpy().flatten().tolist()
        file_id = os.path.basename(path)
        gallery_records.append((file_id, emb))



# Prediction

In [6]:
# -----------------------------------------
# 5) Fetch all embeddings at once
# -----------------------------------------
def load_all_embeddings():
    cur, conn = connect()
    register_vector(conn)
    cur.execute("SELECT file_id, embedding FROM public.fingerprint_embeddings_new ORDER BY file_id ASC;")
    rows = cur.fetchall()
    cur.close()
    conn.close()

    file_ids = [row[0] for row in rows]
    embeddings = np.stack([np.array(row[1], dtype=float) for row in rows], axis=0)
    return file_ids, embeddings

# Usage:
file_ids, embeddings = load_all_embeddings()
print("Loaded", len(file_ids), "embeddings from DB")
gallery_embeddings = np.array(embeddings)  # shape [N,128]


Loaded 6000 embeddings from DB


In [7]:
def predict_fingerprint_from_db(test_image_path, top_k=1):
    img = Image.open(test_image_path).convert("L")
    t = transform(img).unsqueeze(0)  # shape [1,1,224,224]
    t = t.to(device)

    with torch.no_grad():
        q_emb = model(t).cpu().numpy()  # shape [1,128]

    sims = cosine_similarity(q_emb, gallery_embeddings)[0]  # shape [N]
    best_idxs = np.argsort(sims)[::-1][:top_k]

    results = [(file_ids[i], float(sims[i])) for i in best_idxs]
    return results

In [10]:
from sklearn.metrics.pairwise import cosine_similarity

# Update Attendance


In [22]:
from datetime import datetime
import uuid

def mark_attendance_for_best_match():
    subject_code = input("Enter the subject code: ")
    command = input("Enter 'start' to mark attendance, Enter 'close' to stop the attendance: ")

    while command.lower() != "close":
        image_path = input("Enter the path of the fingerprint image: ").strip().strip('"')

        matches = predict_fingerprint_from_db(image_path, top_k=1)

        if not matches:
            print("No match found.")
            continue

        user_id = matches[0][0].split("_")[0]
        print(f"Predicted user ID: {user_id}")
        print("Top matches:")
        for file_id, score in matches:
            print(f"→ {file_id} (similarity: {score:.4f})")

        cur, conn = connect()
        now = datetime.now()
        timestamp = now.strftime("%Y-%m-%d %H:%M:%S")
        attendance_id = str(uuid.uuid4())  # or keep your string format

        # Fetch department and year
        cur.execute("SELECT dept, year FROM users WHERE user_id = %s;", (user_id,))
        user_info = cur.fetchone()
        if not user_info:
            print(f"User ID {user_id} not found in users table.")
            continue

        department, year = user_info

        # Insert attendance record
        cur.execute("""
            INSERT INTO mark_attendance (attendance_id, user_id, department, year, subject_code, date, status)
            VALUES (%s, %s, %s, %s, %s, %s, %s);
        """, (attendance_id, user_id, department, year, subject_code, timestamp, "present"))

        conn.commit()
        print(f"✅ Attendance marked as 'Present' for user_id: {user_id} at {timestamp}")

        command = input("Enter next command ('close' to stop): ")

    conn.close()


In [24]:

mark_attendance_for_best_match()

Predicted user ID: 3
Top matches:
→ 3__M_Left_middle_finger.BMP (similarity: 1.0000)
✅ Attendance marked as 'Present' for user_id: 3 at 2025-05-26 14:00:34
Predicted user ID: 14
Top matches:
→ 14__M_Right_index_finger.BMP (similarity: 1.0000)
✅ Attendance marked as 'Present' for user_id: 14 at 2025-05-26 14:01:09


In [26]:
cur, conn = connect()
cur.execute("SELECT * FROM mark_attendance")
res=cur.fetchall()
for row in res:
    print(row)

('56683055-bba2-485e-b705-c1131e5962ea', 1, 'CE', 2, 'CSE3217', datetime.date(2025, 5, 26), 'present')
('a646a108-75c5-4032-bbe4-1746356477ef', 3, 'CSE', 4, 'CSE3217', datetime.date(2025, 5, 26), 'present')
('7ae7339d-e3bb-467a-9199-95fcc3918149', 14, 'CSE', 4, 'CSE3217', datetime.date(2025, 5, 26), 'present')
